In [1]:
from perceval.backends.core.github import GitHub
from datetime import date
import datetime
import dateutil.tz
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy as s
import augur

In [2]:
if("notebooks" not in os.getcwd()):
    os.chdir("notebooks")
augurApp = augur.Application('../augur.config.json')
connect,list1,path = augurApp.github_issues()
token = list1[5]
DB_STR = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8'.format(
            list1[0], list1[1], list1[2],\
            list1[3], list1[4]
        )
db = s.create_engine(DB_STR)

In [3]:
if "notebooks" in os.getcwd():
    os.chdir("..")
connect,list1,path = augurApp.github_issues()
print(path)
if(not os.path.exists(path)):
    file = open(path, "w+")
else:
    file = open(path, "r")
    print("yeah")
if (os.stat(path).st_size == 0):
    file.write("owner,repo_url\n")
    file.write("nodejs,\"CTC\"\n")
    #file.write("https://lists.opendaylight.org/pipermail/,\"archetypes-dev\"\n")
    print("Please enter the mailing lists and the links for them please")
    print("Going to the default mailing lists")

count = 0
for line in file:
    print(line)
    count+=1
    if(count == 2):
        break

runtime/git_repos.csv
yeah
owner,repo_url

nodejs,"CTC"



In [4]:
# Url for the git repo to analyze
#repo_url = 'https://github.com/chaoss/grimoirelab-perceval'
#repo_url = 'CTC'
# Directory for letting Perceval clone the git repo
#repo_dir = 'CTC'
#token = 'e3f0dd3ccf0174ba4fe351c7c12a275b666f2c34'
#own = 'nodejs'
# ElasticSearch instance (url)
#repo = 'grimoirelab-perceval'

# Create the 'commits' index in ElasticSearch
# Create a Git object, pointing to repo_url, using repo_dir for cloning

#32
#date = datetime.datetime(2017, 8, 7, 0, 0, 0,
#                                     tzinfo=dateutil.tz.tzutc())
k=0
#for issues in repo.fetch(from_date=date):
columns1 = 'augurmsgID',"backend_name",'repo_link',"owner","repo","subject",\
          "status","category","pull_request_number","timestamp",\
          "pull_request_id","user","body"
df = pd.DataFrame(columns=columns1)
item = 1
df.to_sql(name="github_pull_requests", con=db,\
          if_exists='replace',index=False,
           dtype={'augurmsgID': s.types.Integer,#0
                  'backend_name': s.types.VARCHAR(length=300),#1
                  'repo_link': s.types.VARCHAR(length=300),#2
                  'owner': s.types.VARCHAR(length=300),#3
                  'repo': s.types.VARCHAR(length=300),#4
                  'subject': s.types.VARCHAR(length=300),#5
                  'status': s.types.VARCHAR(length=10),#6
                  'category': s.types.VARCHAR(length=10),#7
                  'pull_request_number': s.types.Integer,#8
                  'timestamp': s.types.Integer,#9
                  'pull_request_id': s.types.Integer,#10
                  'user': s.types.VARCHAR(length=100),#11
                  'body':s.types.TEXT#12              
           })

df5 = pd.read_csv(path)
groups = df5.groupby('owner').groups
x = 0
for group in groups:
    own = group
    repo_url = (df5.loc[df5['owner'] == group]['repo_url']).tolist()
    for y in repo_url:        
        repo = GitHub(owner=own,repository=y,api_token=token)
        for issues in repo.fetch():
            # Create the object (dictionary) to upload to ElasticSearch
            # Create the object (dictionary) to upload to ElasticSearch
            #if(i==90):
            if 'pull_request' in issues['data']:
                #print(issues['data']['number'])
                #if(issues['data']['state'] == 'closed' and issues['data']\
                #   ['number'] == 165): 
                text = issues['data']['body']
                user = issues['data']['user']['login']
                num = issues['data']['number']
                id_num = issues['data']['id']
                #print(id_num)
                store = [item,issues['backend_name'],issues['tag'],own,y,\
                         issues['data']['title'],issues['data']['state'],\
                         issues['category'],num,issues['timestamp'],\
                         id_num,user,text]
                #print(store[:])
                #print(store[6])
                #print("Issue number: ",issues['data']['number'])
                #if(k > 1):
                    #k+=1
                    #continue
                '''print("Backend_name: ", issues['backend_name'])
                print("Category: ", issues['category'])
                print("Subject: ", issues['data']['title'])
                print("State: ", issues['data']['state'])
                print("First body:",issues['data']['body'])
                print("User: ", issues['data']['user']['login'])
                print("\n\n\n")'''
                df_user = pd.DataFrame([store],columns=columns1)
                df = df.append(df_user)
                item+=1
                for i in range(len(issues['data']['comments_data'])):
                    temp=0
                    store[0] = item
                    store[8] = issues['data']['number']
                    store[10] = issues['data']['id']
                    store[11] = issues['data']['comments_data'][i]\
                    ['user_data']['login']
                    store[12] = issues['data']['comments_data'][i]['body']
                    df_user = pd.DataFrame([store],columns=columns1)
                    df = df.append(df_user)
                    #print(issues['data']['comments_data'][i]\
                    #      ['user_data']['login'])
                    #print(issues['data']['comments_data'][i]['body'])
                    item+=1

            if(df.shape[0] > 1000):
                df.to_sql(name='github_pull_requests', con=db,\
                          if_exists='append',index=False)
                df = pd.DataFrame(columns=columns1)
                print("Broken")
                #break
            #print(issues['data'][''])
            if(k%50 == 0):
                print(k)
            k+=1
            #if(k == 2):
            #    break
        #print(df)
        #print(df)
        if(df.shape[0] < 1000):
            df.to_sql(name='github_pull_requests', con=db,\
                          if_exists='append',index=False)
            df = pd.DataFrame(columns=columns1)
            print("Broken")
        print("Total Number of pull requests: ", k)
        print(df.shape[0])

2018-07-25 16:02:29 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/GeorgLink


0


2018-07-25 16:02:30 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/germonprez
2018-07-25 16:02:31 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/downeym
2018-07-25 16:02:32 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/aserebrenik
2018-07-25 16:02:32 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/jgbarah
2018-07-25 16:02:33 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/aswanipranjal
2018-07-25 16:02:34 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/jsdabrowski
2018-07-25 16:02:34 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/f2014169
2018-07-25 16:02:35 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/jaishreedhage
2018-07-25 16:02:35 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/

50


2018-07-25 16:02:48 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/sgoggins
2018-07-25 16:02:50 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/geekygirldawn


...

1000


2018-07-25 16:19:14 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/hany0622
2018-07-25 16:19:14 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/lzto
2018-07-25 16:19:15 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/ubante
2018-07-25 16:19:15 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/9577865172
2018-07-25 16:19:16 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/eosrei
2018-07-25 16:19:16 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/crrlos
2018-07-25 16:19:17 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/michaelgeorgeattard
2018-07-25 16:19:17 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/Robdeprop
2018-07-25 16:19:17 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/GijsWe

2018-07-25 16:19:48 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/jdemonasterio
2018-07-25 16:19:49 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/LuyangLi
2018-07-25 16:19:49 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/darrenmhill
2018-07-25 16:19:50 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/wyqwxj
2018-07-25 16:19:50 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/nanszy
2018-07-25 16:19:51 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/peterpansjtu
2018-07-25 16:19:52 keanu-Inspiron-5567 root[5847] INFO Getting info for https://api.github.com/users/xeronith


1050


RateLimitError: Rate limit exhausted.; 1807.0 seconds to rate reset